<a href="https://colab.research.google.com/github/harvardnlp/pytorch-struct/blob/master/notebooks/Unsupervised_CFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qqq torchtext -qqq pytorch-transformers dgl
!pip install -qqqU git+https://github.com/harvardnlp/pytorch-struct

In [0]:
import torchtext
import torch
from torch_struct import SentCFG
from torch_struct.networks import NeuralCFG
import torch_struct.data

In [0]:
# Download and the load default data.
WORD = torchtext.data.Field(include_lengths=True)
UD_TAG = torchtext.data.Field(init_token="<bos>", eos_token="<eos>", include_lengths=True)

# Download and the load default data.
train, val, test = torchtext.datasets.UDPOS.splits(
    fields=(('word', WORD), ('udtag', UD_TAG), (None, None)), 
    filter_pred=lambda ex: 5 < len(ex.word) < 30
)

WORD.build_vocab(train.word, min_freq=3)
UD_TAG.build_vocab(train.udtag)
train_iter = torch_struct.data.TokenBucket(train, 
    batch_size=200,
    device="cuda:0")


In [0]:
H = 256
T = 30
NT = 30
model = NeuralCFG(len(WORD.vocab), T, NT, H)
model.cuda()
opt = torch.optim.Adam(model.parameters(), lr=0.001, betas=[0.75, 0.999])

In [0]:
def train():
    #model.train()
    losses = []
    for epoch in range(10):
        for i, ex in enumerate(train_iter):
            opt.zero_grad()
            words, lengths = ex.word 
            N, batch = words.shape
            words = words.long()
            params = model(words.cuda().transpose(0, 1))
            dist = SentCFG(params, lengths=lengths)
            loss = dist.partition.mean()
            (-loss).backward()
            losses.append(loss.detach())
            torch.nn.utils.clip_grad_norm_(model.parameters(), 3.0)
            opt.step()

            if i % 100 == 1:            
                print(-torch.tensor(losses).mean(), words.shape)
                losses = []

In [0]:
train()

tensor(52.3219) torch.Size([29, 18])


In [0]:
for i, ex in enumerate(train_iter):
    opt.zero_grad()
    words, lengths = ex.word 

    N, batch = words.shape
    words = words.long()
    params = terms(words.transpose(0, 1)), rules(batch), roots(batch)

    tree = CKY(MaxSemiring).marginals(params, lengths=lengths, _autograd=True)
    print(tree)
    break

In [0]:
def split(spans):
    batch, N = spans.shape[:2]
    splits = []
    for b in range(batch):
        cover = spans[b].nonzero()
        left = {i: [] for i in range(N)}
        right = {i: [] for i in range(N)}
        batch_split = {}
        for i in range(cover.shape[0]):
            i, j, A = cover[i].tolist()
            left[i].append((A, j, j - i + 1))
            right[j].append((A, i, j - i + 1))
        for i in range(cover.shape[0]):
            i, j, A = cover[i].tolist()
            B = None
            for B_p, k, a_span in left[i]:
                for C_p, k_2, b_span in right[j]:
                    if k_2 == k + 1 and a_span + b_span == j - i + 1:
                        B, C = B_p, C_p
                        k_final = k
                        break
            if j > i:
                batch_split[(i, j)] =k
        splits.append(batch_split)
    return splits 

In [0]:
splits = split(spans)